# Project

Links:
- https://wandb.ai/andrewkho/wordle-solver/reports/Solving-Wordle-with-Reinforcement-Learning--VmlldzoxNTUzOTc4
- https://github.com/andrewkho/wordle-solver
- https://github.com/kgreed1/wordle-neural-net
- https://www.youtube.com/watch?v=v68zYyaEmEA (3b1b Information theory)

-> stupid idea. can be solved analytically

In [5]:
import random


MAX_ROUNDS = 6

State Vector of size 1+(26\*5\*3) = 391
 * [0] for the remaining turns
 * 5 \* 26 \* 3 for each letter in each position yes/maybe/no
     + [1, 0, 0] - char is definitely not in this spot
     + [0, 1, 0] - char is maybe in this spot
     + [0, 0, 1] - char is definitely in this spot

# Game

In [40]:
import collections
from IPython.display import display
from IPython.display import HTML as html_print
import numpy as np


class Wordle:
    def __init__(self, vocab, max_rounds, solution):
        self.vocab = vocab
        self.max_rounds = max_rounds
        self.round = 1
        self.solution = solution
        self.state = np.zeros(390)
        self.won = False
        self.over = False

    def guess(self, word):
        if self.over:
            return False
        else:
            self.set_state(word.lower())
            return True

    def play_visual(self):
        while True:
            guess = input()
            if guess == "q":
                return
            if len(guess) != 5:
                print("Word must be 5 characters long")
            # TODO: check non ASCII chars
            else:
                if self.guess(guess):
                    self._visualize_state(self.state, guess)
                if self.over:
                    print("Game over, you ", "won!" if self.won else "lost!")
                    print("Solution: ", self.solution)
                    return

    def set_state(self, word):
        reward = 0
        self.round += 1
        if word == self.solution:
            self.won = True
            reward += 50
            self.over = True
        elif self.round > self.max_rounds:
            self.over = True

        solution_char_count = collections.Counter(self.solution)
        for i, char in enumerate(word):
            position_offset = i * 26 * 3
            char_offset = (ord(char) - 97) * 3
            if solution_char_count[char] > 0:
                if word[i] == self.solution[i]:
                    # Character is correct at this position
                    # Set "definitely" on this position (and reset "maybe")
                    self.state[position_offset + char_offset:position_offset + char_offset + 3] = [0, 0, 1]
                    #reward += 10
                else:
                    # character exists in solution and has been seen less often than it exists
                    # Set "maybe" on this position
                    self.state[position_offset + char_offset + 1] = 1
                    #reward += 5
            else:
                if char in self.solution:
                    # character exists in solution but has already been seen as often as it exists
                    # Set "definitely not" on this position, reset "maybe" for all remaining positions
                    self.state[position_offset + char_offset:position_offset + char_offset + 3] = [1, 0, 0]
                    for n in range(i, len(word)):
                        other_positions_offset = n * 26 * 3
                        self.state[other_positions_offset + char_offset + 1] = 0
                else:
                    # character does not exist in solution
                    # Set "definitely not" on all positions
                    for n in range(len(word)):
                        other_positions_offset = n * 26 * 3
                        self.state[other_positions_offset + char_offset:
                                   other_positions_offset + char_offset + 3] = [1, 0, 0]

            solution_char_count[char] -= 1
        return self.state, reward

    def _html_colored(self, text, color="black"):
        return "<text style=background-color:{};font-size:large>{}</text>".format(color, text)

    def _visualize_state(self, state, word):
        colors = np.array(["lightgrey", "yellow", "lightgreen"])
        visual = []
        for i, char in enumerate(word):
            position_offset = i * 26 * 3
            char_offset = (ord(char) - 97) * 3
            position_state = state[position_offset + char_offset:position_offset + char_offset + 3]
            if np.array_equal(position_state, np.zeros(3)):
                color = None
            else:
                color = colors[position_state == 1][0]
            visual.append(self._html_colored(char, color))
        display(html_print("".join(visual)))


In [41]:
vocab_subset_len = 100
random_seed = 69
with open("data/possible_words.txt") as word_list:
    vocab = word_list.read().split('\n')
if vocab_subset_len:
    random.seed(random_seed)
    vocab = random.sample(vocab, k=vocab_subset_len)
random.seed(random_seed)
solution = random.choice(vocab)
game = Wordle(vocab, 6, solution)

In [42]:
game.play_visual()

apple


artsy


Game over, you  won!
Solution:  artsy


state space: 
2k actions you could take at each turn. 
After each of these steps, the game can respond with one of 3^5 combinations of Noes, Maybes, and Yeses. 
So you could think of this as (2k * 3^5)^6 possible sequences

# Model
### -> deep q learning

Reward: 
- +10 for each green 
- +5 for yellow
- (MAX_ROUNDS-n)+1*50 for win DOES NOT WORK!


Now: 
-50 for loose
+50 for win

# Agent